<a href="https://colab.research.google.com/github/btcnhung1299/tf-practice/blob/master/IMG_HorseVsPeople.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q Pillow

In [ ]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf

## Data Gathering

Download dataset and extract to `train_data` and `val_data`.

In [ ]:
!wget http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
!wget http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip

--2020-09-10 16:11:46--  http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.71.128, 74.125.133.128, 74.125.140.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.71.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M  32.0MB/s    in 4.5s    

2020-09-10 16:11:51 (32.0 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]

--2020-09-10 16:11:51--  http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.15.128, 173.194.76.128, 2a00:1450:400c:c07::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.15.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (

In [ ]:
TRAIN_DIR = "./train_data"
VAL_DIR = "./val_data"
!unzip -q horse-or-human.zip -d $TRAIN_DIR
!unzip -q validation-horse-or-human.zip -d $VAL_DIR

Read a sample image and check pixel range.

In [ ]:
train_sample_fn = os.listdir(os.path.join(TRAIN_DIR, "horses"))[0]
train_sample_path = os.path.join(TRAIN_DIR, "horses", train_sample_fn)

**Note:** The last channel is transparency.

In [ ]:
img = Image.open(train_sample_path)
img_data = np.asarray(img)

print("Image shape:", img_data.shape)
print("Pixel in range:", np.min(img_data), np.max(img_data))

print(img_data[:1])

Image shape: (300, 300, 4)
Pixel in range: 1 255
[[[ 34  28  27 255]
  [ 35  29  28 255]
  [ 34  28  27 255]
  ...
  [ 31 110 173 255]
  [ 31 109 172 255]
  [ 34 109 170 255]]]


## Data Flow

In [ ]:
img_size = (150, 150)
img_shape = (*img_size, 3)    # We didn't include transparency channel

Create data flow from a directory, each image is scaled to the range 0-255.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def gen_new_data(data_folder):
  data_gen = ImageDataGenerator(rescale=1./255)
  data = data_gen.flow_from_directory(
      data_folder,
      target_size=img_size,
      batch_size=64,
      class_mode="binary",
  )
  return data

In [ ]:
train_data_gen = gen_new_data(TRAIN_DIR)
val_data_gen = gen_new_data(VAL_DIR)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## Load Pretrained Model

Load InceptionV3 pretrained on ImageNet and freeze all layers above `mixed7`. This layer will give us features extracted from InceptionV3. We're gonna replace the top of the model with our own classifier.

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pretrained_model = InceptionV3(include_top=False, weights="imagenet", input_shape=(150, 150, 3))

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
# Freeze all layers
for layer in pretrained_model.layers:
  layer.trainable = False

# We keep all layers up to `mixed7`
last_pretrained_layer = pretrained_model.get_layer(name="mixed7")
print("Last layer shape:", last_pretrained_layer.output_shape)

Last layer shape: (None, 7, 7, 768)


## Model Architecture & Training

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [ ]:
train_model = Flatten()(last_pretrained_layer.output)
train_model = Dense(1024, activation="swish")(train_model)
train_model = Dropout(0.1)(train_model)
train_model = Dense(1, activation="sigmoid")(train_model)

In [ ]:
model = Model(pretrained_model.input, train_model)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])
model.fit(train_data_gen, epochs=5, validation_data=val_data_gen)

Epoch 1/5
17/17 [==============================] - 9s 508ms/step - loss: 0.1877 - acc: 0.9620 - val_loss: 1.5157e-04 - val_acc: 1.0000
Epoch 2/5
17/17 [==============================] - 7s 440ms/step - loss: 6.3961e-07 - acc: 1.0000 - val_loss: 1.4356e-04 - val_acc: 1.0000
Epoch 3/5
17/17 [==============================] - 7s 438ms/step - loss: 1.7224e-06 - acc: 1.0000 - val_loss: 1.1492e-04 - val_acc: 1.0000
Epoch 4/5
17/17 [==============================] - 7s 434ms/step - loss: 1.2331e-07 - acc: 1.0000 - val_loss: 1.1271e-04 - val_acc: 1.0000
Epoch 5/5
17/17 [==============================] - 7s 437ms/step - loss: 2.6202e-07 - acc: 1.0000 - val_loss: 9.2256e-05 - val_acc: 1.0000


In [ ]:
model.evaluate(val_data_gen)

4/4 [==============================] - 1s 171ms/step - loss: 9.2253e-05 - acc: 1.0000


[9.225314715877175e-05, 1.0]